## Tutorial for `datamanager` Using `ExtArgs`

This tutorial demonstrates use of datamanager using the `ExtArgs` pattern, which is implicitly used when calling datamanager from Snakemake with the `script` syntax or when using the CLI. This is arguably the preferred approach in most cases since it ensures consistency: any problems present using this interface would also appear when interfacing with datamanager from Snakemake or CLI.

### Contents of this tutorial

* [Setup and Overview](#setup-and-overview)
* [Run Management](#run-management)



### Setup and Overview

There is an `ExtArgs` Pydantic model defined in `cmd_interface/external.py`. You should instantiate one of these objects for each "command". Then that object should be passed as the last (third) argument to `entry_point()`.

Due to datamanager not being in a public package directory, we will have to ensure `syspath` allows us to import as expected (it's assumed this notebook is being run from some subpath under the EXIOBASE installation). Along with other standard stuff, this is done below.

In [1]:
import os
import sys
import re
import time
import warnings

from pathlib import Path
from pprint import pprint
from datetime import datetime

# Turn on all warnings... https://docs.python.org/3/library/warnings.html

if not sys.warnoptions:
    warnings.simplefilter("default")  # Change the filter in this process
    os.environ["PYTHONWARNINGS"] = "default"  # Also affect subprocesses

cur_working_dir = os.getcwd()
print(f"Current working directory: {cur_working_dir}")

# Iterate back until we find the EXIOBASE install notice, or we hit the max
# number of allowed iterations.
cur_check_dir = Path(cur_working_dir).resolve()
b_found_install = False
for i in range(0, 15):
    cur_check_dir = cur_check_dir.parent
    if cur_check_dir.is_dir() and Path(cur_check_dir / "exiobase_install_notice.txt").is_file():
        # Found root dir for EXIOBASE install
        b_found_install = True
        print(f"Found EXIOBASE install directory at: {cur_check_dir}")
        break

if not b_found_install:
    raise Exception("Failed to find EXIOBASE installation!")

datamanager_dir = str(Path(cur_check_dir / "repo" / "datamanager").resolve())
if not datamanager_dir in sys.path:
    sys.path.append(datamanager_dir)
# pprint(sys.path)

# The required imports
import datamanager
from datamanager import entry_point
from datamanager.cmd_interface.external import ExtArgs



Current working directory: /home/peterma/exio_dev/repo/docs
Found EXIOBASE install directory at: /home/peterma/exio_dev
'In datamanager/__init__.py, conda env: exio-datamanager'


### Run Management